# Inference for Faster-RCNN

# Setup

Enable GPU acceleration in Google Colab!

In [ ]:
import torch

from IPython.display import Image, clear_output  # to display images
from google.colab.patches import cv2_imshow

clear_output()

print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

## run once to load model

In [ ]:
import torch
import torchvision
import cv2
from torchvision import transforms

device = 'cuda'
imgsz = 416
label_path = label_path='path-to-label-file' # path to file containing class label mapping according to VOC standard

state_dict=torch.load('path-to-weight-file')

# following is state_dict manipulation, as PyTorch Lighning (used for training) does save model state different from regular PyTorch
# prepending 'model.' is removed to load correct weights

for key in list(state_dict.keys()):
  state_dict[key.replace('model.', '')] = state_dict.pop(key)

# load model as usual for PyTorch

model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=False, num_classes=2)
model.load_state_dict(state_dict)
model.to(device)

class_names = [name.strip() for name in open(label_path).readlines()]

transform = transforms.Compose([
 transforms.ToTensor()
])

img = cv2.imread('path-to-image') # image for initial run
img_r = cv2.resize(img, (416,416))

scaley = img.shape[0]/imgsz
scalex = img.shape[1]/imgsz

img_p = transforms.ToPILImage()(img_r)
img_t = transform(img_p)
batch_t = torch.unsqueeze(img_t, 0).to(device)

model.eval()

out = model(batch_t)

print(out)
print(out[0]['boxes'])

In [3]:
def detect(path):
    img = cv2.imread(path)
    img_r = cv2.resize(img, (416,416))

    scaley = img.shape[0]/imgsz
    scalex = img.shape[1]/imgsz

    img_p = transforms.ToPILImage()(img_r)
    img_t = transform(img_p)
    batch_t = torch.unsqueeze(img_t, 0).to(device)

    out = model(batch_t)

    return out, img_r

# Inference on folder


In [ ]:
import os

directory = 'path-to-inference-folder'

for filename in os.listdir(directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):

        path = os.path.join(directory, filename)

        out, img = detect(path)

        for i in range(len(out[0]['boxes'])):
            xyxy = out[0]['boxes'][i].data
            cv2.rectangle(img, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (255, 255, 255), 4)
            label = f'{class_names[out[0]['labels'][i]]}: {out[0]['scores'][i]:.2f}'
            cv2.putText(img, label,
                        (xyxy[0] - 10, xyxy[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,  # font scale
                        (255, 255, 255),
                        2)  # line type

        cv2_imshow(img)